# Static satellite images --> animated gifs

The satellite images are available from [met.no](https://api.met.no/weatherapi/) as part of their API data. This means we can query them with Python and then process them as we wish ([within their guidance](https://api.met.no/doc/TermsOfService)).

First we need to get a list of the images that are available.

In [12]:
# we will use the "requests" package so we need to import it
import requests

# in line with their terms of service, we need to tell met.no who we are. Add your details below...
headers = {
    'User-Agent': 'ADD YOUR NAME HERE',
    'From': 'ADD YOUR EMAIL ADDRESS HERE'  # This is another valid field
}

# next we use the "get" method to retrieve a list of the available images
response = requests.get("https://api.met.no/weatherapi/geosatellite/1.4/available",headers=headers)


In [ ]:
# we can look at the list by seeing what the content of the response is
print(response.content)

What's that format? It's an `xml` file. To work on this in a script, we need a package that will be able to interpret it. 

There is no built-in package in Python to do this so we need to install one. 

Using google we can find some example code to try here: [geeksforgeeks.org](https://www.geeksforgeeks.org/python-xml-to-json/)

In [ ]:
# install the package using

# adding a `!` before the command runs it as if you were directly in a terminal
!pip install xmltodict

In [15]:
# now it's installed we can import it
import xmltodict

# and use it to convert the xml to a dictionary
availableImages = xmltodict.parse(response.content)

In [ ]:
# now let's look at the content again...
availableImages

We can look at this and see that there are different fields before the actual data. The data we want is in a dictionary called 'query' inside another dictionary called 'available'.
To look at it we can use `print(availableImages['available']['query'])`, look at a single item with a `[]` suffix or see how many there are by putting a `len()` around it. Let's look at the first item.

In [ ]:
# that looks complicated, how about we just look at the first item...
print(availableImages['available']['query'][0])

We can see that even within a single item, there are two more dictionaries one of which contain an arrays of dictionaries, the second of which is a single item dictionary. 

This is metadata on each image that is available. 

Using the metadata we can start to categorise each image using loops. But to help we may wish to find out what different values are in each field.

In [ ]:
availableImages['available']['query'][0]

In [19]:
# let's create an empty dictionary to put the lists in
fields = {}

# now we loop through the list of images
for image in availableImages['available']['query']:

    # and within each image we want to look at the parameters
    for detail in image['parameter']:
        
        # if we haven't seen the key before, 
        # add it as an array to the fields dictionary
        # with the value as the first element
        if detail['name'] not in fields.keys():
            fields[detail['name']] = [detail['value']]

        # if we have seen it before, add the value if it's
        # not already in the array
        elif detail['value'] not in fields[detail['name']]:
            fields[detail['name']].append(detail['value'])

    

Now we can look at the different fields either individually or as a group

In [ ]:
print(fields.keys()) # to show all of the keys
print(fields['area']) # to see the values in the 'area' dictionary
print(fields['size']) # to see the values in the 'size' area
print(fields['type']) # to see the values in the 'type' area

We can now start categorising. There are images of:
- Mediterranean
- Europe
- Africa
- Global
- Atlantic_ocean

We have images of either size:
- normal, or
- small

For our task, we'll only want to use the small size images (it's quicker and we're about to hit their server with 30 requests for all their images)

The images are of either the:
- visible, or
- infrared spectrum

All of this information is, of course, available in the [API documentation](https://api.met.no/weatherapi/geosatellite/1.4/documentation). It's a good idea to read the documentation!

If we want to download all of these images, we should put them into a directory structure so that we can easily browse them. For this, we'll need to be able to create directories. We could do this manually, or we could use Python :)

In [22]:

import os

os.mkdir("satelliteImages")
for area in fields['area']:
    os.mkdir(f"./satelliteImages/{area}")
    for spectrum in fields['type']:
        os.mkdir(f"./satelliteImages/{area}/{spectrum}")


If we run the cell above a second time, we get an error. To be robust we should check to see if the directories already exist before we create them with something like `if not os.path.exists(path)`

Next, we want to download all of the images. For this, we're going to have a big loop again...

In [23]:
import shutil
from time import sleep

# here I've restricted the loop to only the first 100 images so it doesn't take too long
# if you want to do it for all images, remove the "[0:100]"
for image in availableImages['available']['query'][0:100]: 
    # for ease let us make some shorthand variable names
    area = image['parameter'][0]['value']
    imageSize = image['parameter'][1]['value']
    imageDate = image['parameter'][2]['value'].replace(":","")
    imageType = image['parameter'][3]['value']
    imageUrl = image['uri']

    # print(imageUrl)
    # now we download the image, but only the larger images...
    if imageSize == "small":

        # if we run this often, we might already have the file, so let's check for that first...
        filename = f'satelliteImages/{area}/{imageType}/{imageDate}.png'
        if not os.path.exists(filename):
            response = requests.get(imageUrl, stream=True,headers=headers)
            # print(response.status_code)

            if response.status_code == 200:                
                with open(filename, 'wb') as out_file:
                    response.raw.decode_content = True
                    shutil.copyfileobj(response.raw, out_file)  
            
            # met.no don't like too many requests at a time so
            # we make our program wait half a second after each download
            sleep(0.55)

With all the files downloaded, we now want to make our animated gif. 

We want to do this for every directory that we have made so we can make a function to avoid having to repeat ourselves...

In [24]:
import glob
from PIL import Image

def makeAnimatedGifFromPngsInDirectory(directory):
    # first we get a list of all the png images in the directory
    # because of the date format, we know they will be returned
    # in date order. This is _handy_!
    pngImages = glob.glob(os.path.join(directory, '*.png'))

    if len(pngImages) > 0:
        # now create one frame for each PNG image
        frames = []

        for i in pngImages:
            new_frame = Image.open(i)
            frames.append(new_frame)

        # Save into a GIF file that loops forever
        animatedGifFilename = f'{directory}/animated.gif'
        frames[0].save(animatedGifFilename, format='GIF',
                    append_images=frames[1:],
                    save_all=True,
                    duration=300, loop=0)
        return animatedGifFilename

In [ ]:
# now we can run our function on each of our directories.
directories = []
for area in fields['area']:
    for spectrum in fields['type']:
        directories.append(f"./satelliteImages/{area}/{spectrum}")

for directory in directories:
    print(makeAnimatedGifFromPngsInDirectory(directory))

How about making it easier to view the images? A web page would be useful for that. So let's make one...

A web page uses HTML to show words and images. We can make a very simple web page...

In [26]:
import datetime
htmlHeader = f"""
<html>
  <head>
    <title>Satellite Images</title>
  </head>
  <body>
  <h1>Satellite images from Met.no, updated {datetime.datetime.now().strftime('%A %d %B %Y, %H:%M:%S')}</h1>
"""
htmlFooter = """
  </body>
</html>
"""
htmlContent = ""

for area in fields['area']:
    for spectrum in fields['type']:
        if os.path.exists(f"satelliteImages/{area}/{spectrum}/animated.gif"):
          htmlContent += f"""
          <h2>{area} image in {spectrum} spectrum</h2>
          <center>
            <img src="{area}/{spectrum}/animated.gif" alt="{area} image in {spectrum} spectrum">
          </center>
          <br />
          """

# now we want to combine our html
overallHtml = htmlHeader + htmlContent + htmlFooter

# then write it to a file
with open("satelliteImages/index.html", "w") as htmlFile:
    htmlFile.write(overallHtml)